Start by taking a look at the branches/columns. The names are self explanatory to some extent.

Events -> rows

Branches -> columns

In [2]:
import ROOT as R
import pandas as pd

# data_files = "~/mc_data_comparison/data/*.root"
# mc_files = "~/mc_data_comparison/montecarlo/tritrig_pulser/*.minidst"

data_files = [
    "~/mc_data_comparison/data/hps_014701_00040.root",
    "~/mc_data_comparison/data/hps_014701_00041.root",
    "~/mc_data_comparison/data/hps_014702_00040.root"
]

mc_files = [
    "~/mc_data_comparison/montecarlo/tritrig_pulser/tritrig_pulser_101_to_110.minidst",
    "~/mc_data_comparison/montecarlo/tritrig_pulser/tritrig_pulser_111_to_120.minidst"
]

df_data = R.RDataFrame("MiniDST", data_files)
df_mc = R.RDataFrame("MiniDST", mc_files)

# Printing basic stuff about the df

# print(df_data.GetColumnNames())
# print(df_mc.GetColumnNames())
print("N events in MC files:", df_mc.Count().GetValue())
print("N events in data files:", df_data.Count().GetValue())

print("Mean of mc_part_z:", df_mc.Mean("mc_part_z").GetValue())
print("Min of mc_part_z:", df_mc.Min("mc_part_z").GetValue())
print("Max of mc_part_z:", df_mc.Max("mc_part_z").GetValue())
print("Number of columns in MC df:", df_mc.GetColumnNames().size())
print("Number of columns in data df:", df_data.GetColumnNames().size())


N events in MC files: 34828
N events in data files: 477974
Mean of mc_part_z: 756.8833474800994
Min of mc_part_z: -304.29082884440976
Max of mc_part_z: 1741.9313334320882
Number of columns in MC df: 192
Number of columns in data df: 122


There are 12 types of branches in real data and 13 types of branches in MC data.
The extra branch is `mc_*` type branches which have the MC truth information.

Note: These branches have the same size for a given event:
- `mc_part_*`
- `mc_score_*`
- `ecal_cluster_*`
- `part_*` 

In [3]:
# Get all unique column types in both dataframes
data_keys = sorted({str(col).split('_')[0] for col in df_data.GetColumnNames()})
mc_keys = sorted({str(col).split('_')[0] for col in df_mc.GetColumnNames()})

print("Column types in data:", data_keys)
print("Column types in MC:", mc_keys)
print("MC - data:", set(mc_keys) - set(data_keys))

Column types in data: ['ecal', 'event', 'ext', 'hodo', 'part', 'rf', 'run', 'svt', 'time', 'track', 'trigger', 'v0']
Column types in MC: ['ecal', 'event', 'ext', 'hodo', 'mc', 'part', 'rf', 'run', 'svt', 'time', 'track', 'trigger', 'v0']
MC - data: {'mc'}


In [12]:
# Taking a look at the MC truth and reconstructed particle columns and their sizes.

row_index = 11  # the event you want to inspect
branch_types = ['part']

for branch_type in branch_types:
    for i in df_mc.GetColumnNames():
        if str(i).startswith(branch_type):
            try:
                arrays = df_mc.Range(row_index, row_index + 1).AsNumpy(columns=[str(i)])
                array = arrays[str(i)][0]
                print(f"{str(i)}: size={len(array)}")
            except RuntimeError:
                print(f"{str(i)}: ERROR")

part_charge: size=8
part_ecal_cluster: size=8
part_energy: size=8
part_goodness_of_pid: size=8
part_lcio_type: size=8
part_mass: size=8
part_pdg: size=8
part_px: size=8
part_py: size=8
part_pz: size=8
part_track: size=8
part_track_chi2: size=8
part_type: size=8


In [13]:
df_mc.Range(5).Display(['mc_part_id','ecal_cluster_energy','part_ecal_cluster','mc_part_pz', 'part_pz']).Print()
# df_mc.Range(5).Display(['mc_part_pz','part_z','ecal_cluster_energy','ecal_cluster_mc_id','mc_part_id']).Print()

+-----+------------+---------------------+-------------------+------------+-----------+
| Row | mc_part_id | ecal_cluster_energy | part_ecal_cluster | mc_part_pz | part_pz   | 
+-----+------------+---------------------+-------------------+------------+-----------+
| 0   | 1821       | 0.737966            | -1                | 0.001789   | 4.050735  | 
|     | 1822       |                     | 0                 | 0.002336   | 0.718979  | 
|     | 1823       |                     |                   | 0.004935   |           | 
|     | 1824       |                     |                   | 0.008373   |           | 
|     | 1825       |                     |                   | 0.043184   |           | 
|     | 1826       |                     |                   | 0.000000   |           | 
|     | 1827       |                     |                   | 0.226381   |           | 
|     | 1828       |                     |                   | 0.053889   |           | 
|     | 1829       |   

*Note (CHECK) *

Each event has multiple clusters. The column `ecal_cluster_mc_id` is the index to mc truth particle arrays like `mc_part_px`, `mc_part_energy` etc. In other words, it indicates which MC particle is responsible for that ECAL cluster. 

`mc_id = -1`  indicates this reconstructed cluster has no clean truth particle behind it.

In [5]:
# Printing a few columns of a certain event to inspect

row_index = 5 # the event
disp = df_mc.Range(row_index, row_index + 1).Display({'part_pz','part_ecal_cluster','ecal_cluster_energy', 'mc_part_id','mc_part_pz'})
disp.Print()

+-----+---------------------+------------+------------+-------------------+----------+
| Row | ecal_cluster_energy | mc_part_id | mc_part_pz | part_ecal_cluster | part_pz  | 
+-----+---------------------+------------+------------+-------------------+----------+
| 5   | 1.401693            | 11112      | 0.001010   | 0                 | 1.397863 | 
|     | 0.887644            | 11113      | 0.002022   | -1                | 1.952644 | 
|     | 0.270223            | 11114      | 1.365055   | -1                | 0.275605 | 
|     |                     | 11115      | 0.001007   | 1                 | 0.884926 | 
|     |                     | 11116      | 0.002080   | 2                 | 0.269475 | 
|     |                     | 11117      | 0.000000   |                   |          | 
|     |                     | 11118      | 0.003705   |                   |          | 
|     |                     | 11119      | 0.001162   |                   |          | 
|     |                     | 1112

In [6]:
# Printing some values from a particular branch of all events that meet a threshold
column_name = 'mc_part_z'
arrays = df_mc.AsNumpy(columns=[column_name])
print ("Total events:", len(arrays[column_name]))

for event_idx, particles in enumerate(arrays[column_name]):
    for p_idx, val in enumerate(particles):
        if abs(val) < 0.1:
            print(f"Event {event_idx}, Particle {p_idx}: {val}")    
        
        
        
# Check if MC particles have any parents. ONly care about the parents.
# Could also check the FEEs, compare only mc and reco that have full FEE energy.


Total events: 34828
Event 27813, Particle 4: -0.07662592575405297


***Histograms***

The general code:
```
H = df.Histo1D(
    (hist_name, hist_title, nbins, xmin, xmax),
    column_name
)
```

In [26]:
# Histogram of added pz distribution of MC truth electrons and positrons
# It is not close to 3.7 GeV as expected. Why?

h_mc_psum = df_mc.Define(
    "mc_primary_psum",
    r"""
    std::vector<double> out;
    double psum = 0.0;
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == 11 || mc_part_pdg[i] == -11
            && mc_part_pz[i] > 0.1
            // && mc_part_parents[i].size() == 0
            // && std::abs(mc_part_z[i]) < 1
        ) {
                double p = std::sqrt(
                mc_part_px[i]*mc_part_px[i] +
                mc_part_py[i]*mc_part_py[i] +
                mc_part_pz[i]*mc_part_pz[i] );
                psum += p;
        }
    }
    return psum;
    """
).Histo1D(
    ("h_mc_psum", "psum_{e^{-}} + psum_{e^{+}}; p_{sum} [GeV]; Counts", 1000, 0, 8),
    "mc_primary_psum"
)


C01 = R.TCanvas("C0001","C0001", 500,500)
h_mc_psum.Draw()
C01.Draw()
C01.SaveAs("/mnt/c/Data/UNH/Research/mc_data_comparison/e+e-psum.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: C0001
Info in <TCanvas::Print>: png file /mnt/c/Data/UNH/Research/mc_data_comparison/e+e-psum.png has been created


As expected, the mean is around 3.8, which is close to the beam energy 3.7GeV.

In [27]:
# Histogram for the z vertex position of MC truth electrons and positrons

h_mc_pz = df_mc.Define(
    "mc_z",
    r"""
    std::vector<double> out;
    int countE = 0;
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == 11 || mc_part_pdg[i] == -11
            && mc_part_pz[i] > 0.1
            // && mc_part_parents[i].size() == 0
            // && std::abs(mc_part_z[i]) < 0.01
        )
            out.push_back(mc_part_z[i]);
    }
    return out;
    """
).Histo1D(
    ("h_mc_z", " z_{e^{-}} and z_{e^{+}}; z [mm]; Counts", 50, -5, 5),
    "mc_z"
)


C01 = R.TCanvas("C0001","C0001", 500,500)
h_mc_pz.Draw()
C01.Draw()
C01.SaveAs("/mnt/c/Data/UNH/Research/mc_data_comparison/z_pos_cut.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: C0001
Info in <TCanvas::Print>: png file /mnt/c/Data/UNH/Research/mc_data_comparison/z_pos_cut.png has been created


In [29]:
#####################################################
# Here I am creating histograms for reco and MC truth
# electrons and positrons pz, wihtout any filters.
#####################################################

# Reco particles
# Create histogram for reco electron pz
h_electron_pz = df_mc.Define(
    'electron_pz',
    'vector<double> out; \
     for(int i=0;i<part_pdg.size();++i){ \
         if(part_pdg[i]==11){ \
             out.push_back(part_pz[i]); \
         }; \
     }; \
     return out;'
).Histo1D(
    ('h_electron_pz','electron pz',1000,0,8),
    'electron_pz'
)

# Create histogram for reco positron pz
h_positron_pz = df_mc.Define(
    'positron_pz',
    'vector<double> out; \
     for(int i=0;i<part_pdg.size();++i){ \
         if(part_pdg[i]==-11){ \
             out.push_back(part_pz[i]); \
         }; \
     }; \
     return out;'
).Histo1D(
    ('h_positron_pz','positron pz',1000,0,8),
    'positron_pz'
)

############################################

# MC truth particles
# Create histogram for MC truth electron pz
h_mc_electron_pz = df_mc.Define(
    "mc_electron_pz",
    r"""
    std::vector<double> out;
    out.reserve(mc_part_pdg.size());
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == 11) {
            out.push_back(mc_part_pz[i]);
        }
    }
    return out;
    """
).Histo1D(
    ("h_mc_electron_pz", "MC electron p_{z}; p_{z} [GeV]; Counts", 1000, 0, 8),
    "mc_electron_pz"
)

# MC truth positron pz
h_mc_positron_pz = df_mc.Define(
    "mc_positron_pz",
    r"""
    std::vector<double> out;
    out.reserve(mc_part_pdg.size());
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == -11) {
            out.push_back(mc_part_pz[i]);
        }
    }
    return out;
    """
).Histo1D(
    ("h_mc_positron_pz", "MC positron p_{z}; p_{z} [GeV]; Counts", 1000, 0, 8),
    "mc_positron_pz"
)


In [ ]:
########### Plotting ############

C01 = R.TCanvas("C01", "Reco vs Truth e^+ e^- p_{z}", 650, 500)

# Materialize (convert RResultPtr -> TH1 pointers)
h_reco_e = h_electron_pz.GetPtr()
h_reco_p = h_positron_pz.GetPtr()
h_mc_e   = h_mc_electron_pz.GetPtr()
h_mc_p   = h_mc_positron_pz.GetPtr()

# Styling: colors
h_reco_e.SetLineColor(R.kBlue)
h_reco_p.SetLineColor(R.kRed)
h_mc_e.SetLineColor(R.kBlack)
h_mc_p.SetLineColor(R.kGreen)

# Styling: widths
for h in (h_reco_e, h_reco_p, h_mc_e, h_mc_p):
    h.SetLineWidth(2)

# Styling: line styles (dashed for truth)
h_mc_e.SetLineStyle(2)  # dashed
h_mc_p.SetLineStyle(2)  # dashed

# Title/axes (set on the one you draw first)
h_reco_p.SetTitle("p_{z} distributions: reco vs MC truth; p_{z} [GeV]; Counts")

# Normalize
def norm(h):
    integ = h.Integral()
    if integ > 0:
        h.Scale(1.0/integ)

# If you want raw counts
norm(h_reco_e); norm(h_reco_p); norm(h_mc_e); norm(h_mc_p)


# Draw
h_reco_p.Draw("hist")
h_reco_e.Draw("hist same")
h_mc_p.Draw("hist same")
h_mc_e.Draw("hist same")

# Legend
legend = R.TLegend(0.55, 0.70, 0.88, 0.88)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.03)
legend.AddEntry(h_reco_e, "Reco e^{-}", "l")
legend.AddEntry(h_reco_p, "Reco e^{+}", "l")
legend.AddEntry(h_mc_e,   "MC truth e^{-}", "l")
legend.AddEntry(h_mc_p,   "MC truth e^{+}", "l")
legend.Draw()

C01.Draw()
C01.SaveAs("pz_reco_vs_truth_e+e-.png")


: 

In [10]:
##### Select #####
# start at z=0
# 0.1 GeV cut on pz positive
# No parent
# Check the same plot for MC particles
# Compare only those with full FEE energy mc and & reco

# Once the MC is done, compare it with the reco data.

# After that, do the matching using some filters and add it to a new column, and do further analysis. 

# Histograms should match, if not something went wrong
# Do the difference histogram.

Now, I wanna add some filters and see the data.

- Start at z=0
- No parents
- FEE (energy filters)

I'm gonna do it for MC first and then reco, and then compare them both.

In [23]:
df_mc_counted = df_mc.Define(
    "mc_electron_count",
    r"""
    int count = 0;
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == 11 
            && mc_part_pz[i] > 0.1
            // && mc_part_parents[i].size() == 0
            && mc_part_z[i] < -0.45
            && mc_part_z[i] > -0.50) {
                count++;
            }
    }
    // std::cout << "Found " << count << " electrons in this event" << std::endl;
    return count;
    """
)

# Check how many events have at least one electron
events_with_electrons = df_mc_counted.Filter("mc_electron_count > 0").Count().GetValue()
total_electrons = df_mc_counted.Sum("mc_electron_count").GetValue()

print(f"Events with electrons: {events_with_electrons}")
print(f"Total electrons: {int(total_electrons)}")

Events with electrons: 1517122
Total electrons: 2704891


In [ ]:
## Multiple histogram overlay plotting function

# def plot_histograms(histograms, canvas_name="c", canvas_title="Histogram Comparison",
#                     canvas_width=700, canvas_height=550, normalize=True,
#                     legend_coords=(0.55, 0.70, 0.88, 0.88), legend_title=None):
#
#     canvas = R.TCanvas(canvas_name, canvas_title, canvas_width, canvas_height)
#     canvas.SetLeftMargin(0.12); canvas.SetRightMargin(0.05)
#     canvas.SetTopMargin(0.08);  canvas.SetBottomMargin(0.12)

#     styled = []
#     for h, label, color, linestyle in histograms:
#         h.SetLineColor(color)
#         h.SetLineWidth(2)
#         h.SetLineStyle(linestyle)
#         if normalize and h.Integral() > 0:
#             h.Scale(1.0 / h.Integral())
#         styled.append((h, label))

#     if not styled:
#         return canvas, []

#     # autoscale y using the maximum among histograms
#     ymax = max(h.GetMaximum() for h, _ in styled)
#     if ymax > 0:
#         first = styled[0][0]
#         first.SetMaximum(ymax * 1.15)
#         first.SetMinimum(0)
#         first.GetYaxis().SetTitleOffset(1.2)

#     for i, (h, label) in enumerate(styled):
#         draw_opt = "hist" if i == 0 else "hist same"
#         h.Draw(draw_opt)

#     # legend
#     legend = R.TLegend(*legend_coords)
#     if legend_title:
#         legend.SetHeader(legend_title)
#     legend.SetBorderSize(0)
#     legend.SetFillStyle(0)
#     legend.SetTextSize(0.03)
#     for h, label in styled:
#         legend.AddEntry(h, label, "l")
#     legend.Draw()

#     canvas.Modified(); canvas.Update()
#     return canvas, [h for h, _ in styled]

In [3]:
# MC truth particles
# Create histogram for primary MC truth electron pz

h_mc_primary_electron_pz = df_mc.Define(
    "mc_primary_electron_pz",
    r"""
    std::vector<double> out;
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == 11 
            && mc_part_pz[i] > 0.1
            // && mc_part_parents[i].size() == 0
            && mc_part_z[i] < -0.45
            && mc_part_z[i] > -0.50
            )
        {
            // out.push_back(mc_part_pz[i]); // Just pz
            out.push_back(std::sqrt(mc_part_px[i]*mc_part_px[i] + mc_part_py[i]*mc_part_py[i] + mc_part_pz[i]*mc_part_pz[i])); // Psum
        }
    }
    return out;
    """
).Histo1D(
    ("h_mc_primary_electron_pz", "Primary e^{-} p_{z};p_{z} [GeV];Counts", 1000, 0, 8),
    "mc_primary_electron_pz"
)


# Create histogram for primary MC positrons pz
h_mc_primary_positron_pz = df_mc.Define(
    "mc_primary_positron_pz",
    r"""
    std::vector<double> out;
    for (size_t i = 0; i < mc_part_pdg.size(); ++i) {
        if (mc_part_pdg[i] == -11 
            && mc_part_pz[i] > 0.1
            // && mc_part_parents[i].size() == 0
            && mc_part_z[i] < -0.45
            && mc_part_z[i] > -0.50
            )
            // out.push_back(mc_part_pz[i]); // Just pz
            out.push_back(std::sqrt(mc_part_px[i]*mc_part_px[i] + mc_part_py[i]*mc_part_py[i] + mc_part_pz[i]*mc_part_pz[i])); // Psum
    }
    return out;
    """
).Histo1D(
    ("h_mc_primary_positron_pz", "Primary e^{+} p_{z};p_{z} [GeV];Counts", 1000, 0, 8),
    "mc_primary_positron_pz"
)             


# Reco: primary electron/positron pz with same filters (energy & z)
h_reco_primary_electron_pz = df_mc.Define(
    "reco_primary_electron_pz",
    r"""
    std::vector<double> out;
    out.reserve(part_pz.size());
    for (size_t i = 0; i < part_pdg.size(); ++i) {
        if (part_pdg[i] == 11
            && part_pz[i] > 0.1
            && part_ecal_cluster[i] >= 0
            )
            
            // out.push_back(part_pz[i]); // Just pz
            out.push_back(std::sqrt(part_px[i]*part_px[i] + part_py[i]*part_py[i] + part_pz[i]*part_pz[i])); // Psum
    }
    return out;
    """
).Histo1D(("h_reco_primary_electron_pz", "Reco primary e^{-} p_{z}; p_{z} [GeV]; Counts", 1000, 0, 8),
          "reco_primary_electron_pz")

h_reco_primary_positron_pz = df_mc.Define(
    "reco_primary_positron_pz",
    r"""
    std::vector<double> out;
    out.reserve(part_pz.size());
    for (size_t i = 0; i < part_pdg.size(); ++i) {
        if (part_pdg[i] == -11
            && part_pz[i] > 0.1
            && part_ecal_cluster[i] >= 0
            )
            
            // out.push_back(part_pz[i]); // Just pz
            out.push_back(std::sqrt(part_px[i]*part_px[i] + part_py[i]*part_py[i] + part_pz[i]*part_pz[i])); // Psum
    }
    return out;
    """
).Histo1D(("h_reco_primary_positron_pz", "Reco primary e^{+} p_{z}; p_{z} [GeV]; Counts", 1000, 0, 8),
          "reco_primary_positron_pz")




# # Plot using the function for multiple plots

# # Materialize the histograms
# h_mc_e   = h_mc_primary_electron_pz.GetPtr()
# h_mc_p   = h_mc_primary_positron_pz.GetPtr()
# h_reco_e = h_reco_primary_electron_pz.GetPtr()
# h_reco_p = h_reco_primary_positron_pz.GetPtr()

# # Call the function
# canvas, hists = plot_histograms(
#     [
#         (h_mc_e, "MC truth e^{-}", R.kBlack, 2),
#         (h_mc_p, "MC truth e^{+}", R.kGreen, 2),
#         (h_reco_p, "Reco primary e^{+}", R.kRed, 1),
#         (h_reco_e, "Reco primary e^{-}", R.kBlue, 1),

#     ], legend_coords=(0.55, 0.70, 0.88, 0.88),
#     canvas_name="C02",
#     canvas_title="Primary e^+ e^- p_{z} from MC and Reco",
#     canvas_width=650,
#     canvas_height=500,
#     normalize=True
# )

In [4]:
########### Plotting ############

C01 = R.TCanvas("C01", "Reco vs Truth e^+ e^- p_{z}", 650, 500)

# Materialize the histograms
h_mc_e   = h_mc_primary_electron_pz.GetPtr()
h_mc_p   = h_mc_primary_positron_pz.GetPtr()
h_reco_e = h_reco_primary_electron_pz.GetPtr()
h_reco_p = h_reco_primary_positron_pz.GetPtr()

# Styling: colors
h_reco_e.SetLineColor(R.kBlue)
h_reco_p.SetLineColor(R.kRed)
h_mc_e.SetLineColor(R.kBlack)
h_mc_p.SetLineColor(R.kGreen)

# Styling: widths
for h in (h_reco_e, h_reco_p, h_mc_e, h_mc_p):
    h.SetLineWidth(2)

# Styling: line styles (dashed for truth)
h_mc_e.SetLineStyle(2)  # dashed
h_mc_p.SetLineStyle(2)  # dashed

# Title/axes (set on the one you draw first)
h_reco_p.SetTitle("Primary p_{z} distributions: reco vs MC truth; p_{z} [GeV]; Counts")

# Normalize
def norm(h):
    integ = h.Integral()
    if integ > 0:
        h.Scale(1.0/integ)
norm(h_reco_e); norm(h_reco_p); norm(h_mc_e); norm(h_mc_p)

# Set y-axis range to fit ALL histograms
maxy = max(h.GetMaximum() for h in (h_reco_e, h_reco_p, h_mc_e, h_mc_p))
h_reco_p.SetMaximum(1.15 * maxy)

# Draw
h_reco_p.Draw("hist")
h_reco_e.Draw("hist same")
h_mc_p.Draw("hist same")
h_mc_e.Draw("hist same")

# Legend
legend = R.TLegend(0.55, 0.70, 0.88, 0.88)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.03)
legend.AddEntry(h_reco_e, "Reco e^{-}", "l")
legend.AddEntry(h_mc_e,   "MC truth e^{-}", "l")
legend.AddEntry(h_reco_p, "Reco e^{+}", "l")
legend.AddEntry(h_mc_p,   "MC truth e^{+}", "l")
legend.Draw()

C01.Draw()
# C01.SaveAs("pz_reco_vs_truth_e_pm.png")



***Total Momentum***

Real data seem to have way more particles with 0-3 GeV as compared to MC reconstructed. After that, MC and data are in agreement.

In [3]:
# Total momentum


def add_part_p(df):
    return df.Define(
        "part_p",
        """
        std::vector<double> out;
        out.reserve(part_pz.size());
        for (size_t i = 0; i < part_pz.size(); ++i)
            if (part_pdg[i] == 11
            && part_pz[i] > 0.1
            && part_ecal_cluster[i] >= 0
            )
            {
                // out.push_back(part_pz[i]);
                out.push_back(std::sqrt(part_px[i]*part_px[i] + part_py[i]*part_py[i] + part_pz[i]*part_pz[i]));
            }
        return out;
        """
    )

df_mc2   = add_part_p(df_mc)
df_data2 = add_part_p(df_data)

h_mc_e   = df_mc2.Histo1D(("h_part_p_mc",   "part total p;|p| [GeV];Counts", 1000, 0, 8), "part_p")
h_data_e = df_data2.Histo1D(("h_part_p_data","part total p;|p| [GeV];Counts", 1000, 0, 8), "part_p")



In [4]:
########### Plotting ############

C01 = R.TCanvas("C01", "Reco vs RealData e^- p_{z}", 650, 500)

# Materialize the histograms
h_mc   = h_mc_e.GetPtr()
h_data = h_data_e.GetPtr()

# Styling: colors
h_data.SetLineColor(R.kBlue)
h_mc.SetLineColor(R.kRed)

# Styling: widths
for h in (h_data, h_mc):
    h.SetLineWidth(2)

# Styling: line styles (dashed for truth)
h_mc.SetLineStyle(2)  # dashed
h_data.SetLineStyle(2)  # dashed

# Title/axes (set on the one you draw first)
h_data.SetTitle("Primary psum distributions: reco vs real data; psum [GeV]; Counts")
# Normalize
def norm(h):
    integ = h.Integral()
    if integ > 0:
        h.Scale(1.0/integ)
norm(h_data); norm(h_mc)

# Set y-axis range to fit ALL histograms
maxy = max(h.GetMaximum() for h in (h_data, h_mc))
h_data.SetMaximum(1.15 * maxy)
# Draw
h_data.Draw("hist")
h_mc.Draw("hist same")

# Legend
legend = R.TLegend(0.55, 0.70, 0.88, 0.88)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.03)
legend.AddEntry(h_data, "Real data", "l")
legend.AddEntry(h_mc,   "MC truth", "l")
legend.Draw()

C01.Draw()
# C01.SaveAs("pz_reco_vs_truth_e_pm.png")

